# Entraînement du Modèle Morningstar (Version Locale)

Ce notebook entraîne le modèle `enhanced_hybrid_model` localement en utilisant les données préparées pour un actif spécifique (par défaut 'sol').

In [1]:
# Configuration de base
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path('/home/morningstar/Desktop/crypto_robot/Morningstar')
os.chdir(PROJECT_ROOT)
print(f"Répertoire projet: {PROJECT_ROOT}")

Répertoire projet: /home/morningstar/Desktop/crypto_robot/Morningstar


In [2]:
# Installation des dépendances (si nécessaire)
# !pip install -r requirements.txt

In [3]:
# Chargement des données et préparation des labels
import tensorflow as tf
import numpy as np
import pandas as pd
from model.training.data_loader import load_and_split_data
from model.architecture.enhanced_hybrid_model import build_enhanced_hybrid_model

ASSET_NAME = 'sol'
DATA_PATH = PROJECT_ROOT / 'data' / 'processed' / f'{ASSET_NAME}_final.parquet'

# Vérification des colonnes disponibles
print("Colonnes disponibles dans le fichier:")
print(pd.read_parquet(DATA_PATH).columns.tolist())

# Chargement des données brutes
data = pd.read_parquet(DATA_PATH)

# Renommage des colonnes pour correspondre aux noms de sortie du modèle
data = data.rename(columns={
    'trading_signal': 'signal',
    'volatility': 'volatility_quantiles',
    'market_regime': 'market_regime'
})

# Création de volatility_regime si nécessaire
if 'volatility_regime' not in data.columns:
    data['volatility_regime'] = data['market_regime']  # Utiliser market_regime comme placeholder

# Conversion des régimes textuels en entiers
regime_map = {'bearish_placeholder': 0, 'sideways_placeholder': 1, 'bullish_placeholder': 2}
if 'market_regime' in data.columns and data['market_regime'].dtype == 'object':
    data['market_regime'] = data['market_regime'].map(regime_map).fillna(-1).astype(int)
if 'volatility_regime' in data.columns and data['volatility_regime'].dtype == 'object':
    data['volatility_regime'] = data['volatility_regime'].map(regime_map).fillna(-1).astype(int)

# Préparation de sl_tp à partir de level_sl et level_tp
if 'level_sl' in data.columns and 'level_tp' in data.columns:
    data['level_sl'] = pd.to_numeric(data['level_sl'], errors='coerce').fillna(0)
    data['level_tp'] = pd.to_numeric(data['level_tp'], errors='coerce').fillna(0)
else:
    print("Warning: Colonnes level_sl ou level_tp manquantes")

# Sauvegarde temporaire pour charger avec data_loader
temp_path = PROJECT_ROOT / 'data' / 'processed' / 'temp.parquet'
data.to_parquet(temp_path)

# Chargement avec la nouvelle version de data_loader
output_names = ['signal', 'volatility_quantiles', 'volatility_regime', 'market_regime', 'sl_tp']
X, y_dict = load_and_split_data(
    file_path=temp_path,
    label_columns=output_names,
    as_tensor=True
)

# Suppression du fichier temporaire
temp_path.unlink()

2025-04-18 14:18:24.600403: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-18 14:18:24.666457: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-18 14:18:24.948566: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-18 14:18:24.950548: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2025-04-18 14:18:26.513210: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Colonnes disponibles dans le fichier:
['open', 'high', 'low', 'close', 'volume', 'SMA_short', 'SMA_long', 'EMA_short', 'EMA_long', 'RSI', 'MACD', 'MACDs', 'MACDh', 'BBU', 'BBM', 'BBL', 'ATR', 'STOCHk', 'STOCHd', 'trading_signal', 'volatility', 'level_sl', 'level_tp', 'market_regime', 'dummy_placeholder_1', 'dummy_placeholder_2', 'dummy_placeholder_3', 'dummy_placeholder_4', 'dummy_placeholder_5', 'dummy_placeholder_6', 'dummy_placeholder_7', 'dummy_placeholder_8', 'dummy_placeholder_9', 'dummy_placeholder_10', 'dummy_placeholder_11', 'dummy_placeholder_12', 'dummy_placeholder_13', 'dummy_placeholder_14']


INFO: Label 'sl_tp' créé en empilant 'level_sl' et 'level_tp'.


In [4]:
# Séparation train/validation
num_samples = X.shape[0]
num_val = int(num_samples * 0.2)
X_train, X_val = X[:num_samples-num_val], X[num_samples-num_val:]
y_train = {k: v[:num_samples-num_val] for k, v in y_dict.items()}
y_val = {k: v[num_samples-num_val:] for k, v in y_dict.items()}

In [5]:
# Préparation des inputs pour modèle hybride (avec placeholder pour LLM)
print("\nPréparation des dictionnaires d'input pour le modèle hybride...")
llm_embedding_dim = 768 # Doit correspondre à la définition du modèle

llm_placeholder_train = np.zeros((X_train.shape[0], llm_embedding_dim), dtype=np.float32)
X_train_dict = {
    'technical_input': X_train,
    'llm_input': llm_placeholder_train
}

llm_placeholder_val = np.zeros((X_val.shape[0], llm_embedding_dim), dtype=np.float32)
X_val_dict = {
    'technical_input': X_val,
    'llm_input': llm_placeholder_val
}
print("Dictionnaires d'inputs créés.")


Préparation des dictionnaires d'input pour le modèle hybride...
Dictionnaires d'inputs créés.


In [6]:
# Initialisation du modèle
input_shape = (X_train.shape[1],)
# Déterminer dynamiquement le nombre de classes si possible
num_signal_classes = int(tf.reduce_max(y_train['signal'])) + 1 if 'signal' in y_train else 2
num_market_regime_classes = int(tf.reduce_max(y_train['market_regime'])) + 1 if 'market_regime' in y_train else 3

model = build_enhanced_hybrid_model(
    input_shape=input_shape,
    llm_embedding_dim=llm_embedding_dim,
    num_trading_classes=num_signal_classes,
    num_regime_classes=num_market_regime_classes
)

# Configuration de la compilation
losses = {
    'signal': 'sparse_categorical_crossentropy',
    'volatility_quantiles': 'mse',
    'volatility_regime': 'sparse_categorical_crossentropy',
    'market_regime': 'sparse_categorical_crossentropy',
    'sl_tp': 'mse'
}

metrics = {
    'signal': ['accuracy'],
    'volatility_quantiles': ['mae'],
    'volatility_regime': ['accuracy'],
    'market_regime': ['accuracy'],
    'sl_tp': ['mae']
}

model.compile(
    optimizer='adam',
    loss=losses,
    metrics=metrics
)

In [7]:
# Entraînement du modèle
EPOCHS = 50
BATCH_SIZE = 32
MODEL_PATH = PROJECT_ROOT / 'model' / 'training' / 'morningstar_local.h5'

history = model.fit(
    x=X_train_dict,  # Utiliser le dictionnaire d'inputs
    y=y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val_dict, y_val), # Utiliser les dictionnaires pour la validation aussi
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            filepath=MODEL_PATH,
            save_best_only=True,
            monitor='val_loss'
        ),
        tf.keras.callbacks.EarlyStopping(
            patience=10,
            restore_best_weights=True
        )
    ]
)

Epoch 1/50


  1/220 [..............................] - ETA: 15:30 - loss: 7.6817 - signal_loss: 1.2382 - volatility_quantiles_loss: 0.4943 - volatility_regime_loss: 0.9527 - market_regime_loss: 1.5691 - sl_tp_loss: 3.4275 - signal_accuracy: 0.2500 - volatility_quantiles_mae: 0.4265 - volatility_regime_accuracy: 0.6562 - market_regime_accuracy: 0.1875 - sl_tp_mae: 1.7948

 13/220 [>.............................] - ETA: 0s - loss: 3.9704 - signal_loss: 0.5935 - volatility_quantiles_loss: 0.2841 - volatility_regime_loss: 0.5168 - market_regime_loss: 0.8450 - sl_tp_loss: 1.7310 - signal_accuracy: 0.7740 - volatility_quantiles_mae: 0.3327 - volatility_regime_accuracy: 0.9135 - market_regime_accuracy: 0.6587 - sl_tp_mae: 1.1223   

 25/220 [==>...........................] - ETA: 0s - loss: 2.9373 - signal_loss: 0.3813 - volatility_quantiles_loss: 0.2444 - volatility_regime_loss: 0.3612 - market_regime_loss: 0.5565 - sl_tp_loss: 1.3938 - signal_accuracy: 0.8825 - volatility_quantiles_mae: 0.3014 - volatility_regime_accuracy: 0.9550 - market_regime_accuracy: 0.8225 - sl_tp_mae: 0.9810

 38/220 [====>.........................] - ETA: 0s - loss: 2.2306 - signal_loss: 0.2714 - volatility_quantiles_loss: 0.2260 - volatility_regime_loss: 0.2684 - market_regime_loss: 0.4000 - sl_tp_loss: 1.0647 - signal_accuracy: 0.9227 - volatility_quantiles_mae: 0.2928 - volatility_regime_accuracy: 0.9704 - market_regime_accuracy: 0.8832 - sl_tp_mae: 0.8140

...

182/220 [=======================>......] - ETA: 0s - loss: 0.0075 - signal_loss: 3.9446e-05 - volatility_quantiles_loss: 9.7623e-04 - volatility_regime_loss: 3.7290e-05 - market_regime_loss: 3.8625e-05 - sl_tp_loss: 0.0064 - signal_accuracy: 1.0000 - volatility_quantiles_mae: 0.0227 - volatility_regime_accuracy: 1.0000 - market_regime_accuracy: 1.0000 - sl_tp_mae: 0.0626

194/220 [=========================>....] - ETA: 0s - loss: 0.0075 - signal_loss: 3.8881e-05 - volatility_quantiles_loss: 9.5932e-04 - volatility_regime_loss: 3.6691e-05 - market_regime_loss: 3.8124e-05 - sl_tp_loss: 0.0064 - signal_accuracy: 1.0000 - volatility_quantiles_mae: 0.0225 - volatility_regime_accuracy: 1.0000 - market_regime_accuracy: 1.0000 - sl_tp_mae: 0.0627

206/220 [===========================>..] - ETA: 0s - loss: 0.0075 - signal_loss: 3.8940e-05 - volatility_quantiles_loss: 9.5651e-04 - volatility_regime_loss: 3.6717e-05 - market_regime_loss: 3.8057e-05 - sl_tp_loss: 0.0064 - signal_accuracy: 1.0000 - volatility_quantiles_mae: 0.0224 - volatility_regime_accuracy: 1.0000 - market_regime_accuracy: 1.0000 - sl_tp_mae: 0.0626

218/220 [============================>.] - ETA: 0s - loss: 0.0075 - signal_loss: 3.8954e-05 - volatility_quantiles_loss: 9.3828e-04 - volatility_regime_loss: 3.6598e-05 - market_regime_loss: 3.8010e-05 - sl_tp_loss: 0.0064 - signal_accuracy: 1.0000 - volatility_quantiles_mae: 0.0222 - volatility_regime_accuracy: 1.0000 - market_regime_accuracy: 1.0000 - sl_tp_mae: 0.0624

220/220 [==============================] - 1s 5ms/step - loss: 0.0074 - signal_loss: 3.8868e-05 - volatility_quantiles_loss: 9.3548e-04 - volatility_regime_loss: 3.6535e-05 - market_regime_loss: 3.7921e-05 - sl_tp_loss: 0.0064 - signal_accuracy: 1.0000 - volatility_quantiles_mae: 0.0222 - volatility_regime_accuracy: 1.0000 - market_regime_accuracy: 1.0000 - sl_tp_mae: 0.0624 - val_loss: 484.0432 - val_signal_loss: 1.6064 - val_volatility_quantiles_loss: 98.4120 - val_volatility_regime_loss: 1.3237e-04 - val_market_regime_loss: 2.4987 - val_sl_tp_loss: 381.5258 - val_signal_accuracy: 0.9789 - val_volatility_quantiles_mae: 0.8530 - val_volatility_regime_accuracy: 1.0000 - val_market_regime_accuracy: 0.9738 - val_sl_tp_mae: 1.9587


In [8]:
# Évaluation et visualisation des résultats
results = model.evaluate(X_val_dict, y_val) # Utiliser le dictionnaire pour l'évaluation
print("Résultats d'évaluation:")
for name, value in zip(model.metrics_names, results):
    print(f"{name}: {value:.4f}")

 1/55 [..............................] - ETA: 1s - loss: 0.1619 - signal_loss: 0.0023 - volatility_quantiles_loss: 0.0916 - volatility_regime_loss: 0.0043 - market_regime_loss: 0.0028 - sl_tp_loss: 0.0609 - signal_accuracy: 1.0000 - volatility_quantiles_mae: 0.1991 - volatility_regime_accuracy: 1.0000 - market_regime_accuracy: 1.0000 - sl_tp_mae: 0.1345

20/55 [=========>....................] - ETA: 0s - loss: 0.0969 - signal_loss: 0.0025 - volatility_quantiles_loss: 0.0533 - volatility_regime_loss: 0.0054 - market_regime_loss: 0.0035 - sl_tp_loss: 0.0323 - signal_accuracy: 1.0000 - volatility_quantiles_mae: 0.1751 - volatility_regime_accuracy: 1.0000 - market_regime_accuracy: 1.0000 - sl_tp_mae: 0.1240

38/55 [===================>..........] - ETA: 0s - loss: 0.1083 - signal_loss: 0.0025 - volatility_quantiles_loss: 0.0425 - volatility_regime_loss: 0.0054 - market_regime_loss: 0.0081 - sl_tp_loss: 0.0498 - signal_accuracy: 1.0000 - volatility_quantiles_mae: 0.1408 - volatility_regime_accuracy: 1.0000 - market_regime_accuracy: 0.9992 - sl_tp_mae: 0.1445

55/55 [==============================] - 0s 3ms/step - loss: 4.3098 - signal_loss: 0.0679 - volatility_quantiles_loss: 0.5476 - volatility_regime_loss: 0.0050 - market_regime_loss: 0.0715 - sl_tp_loss: 3.6178 - signal_accuracy: 0.9960 - volatility_quantiles_mae: 0.2243 - volatility_regime_accuracy: 1.0000 - market_regime_accuracy: 0.9880 - sl_tp_mae: 0.3764


Résultats d'évaluation:
loss: 4.3098
signal_loss: 0.0679
volatility_quantiles_loss: 0.5476
volatility_regime_loss: 0.0050
market_regime_loss: 0.0715
sl_tp_loss: 3.6178
signal_accuracy: 0.9960
volatility_quantiles_mae: 0.2243
volatility_regime_accuracy: 1.0000
market_regime_accuracy: 0.9880
sl_tp_mae: 0.3764
